In [1]:
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
conn = wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:

query_symbols = "SELECT DISTINCT SYMBOL FROM optionm.opprcd2022"
symbols = conn.raw_sql(query_symbols)

In [6]:

ice_permno = conn.raw_sql("""
    SELECT DISTINCT permno
    FROM crsp.msenames
    WHERE ticker = 'ICE'
      AND namedt <= '2023-12-31'
      AND nameendt >= '2021-01-01'
      AND shrcd IN (10, 11)        -- common stocks
      AND exchcd BETWEEN 1 AND 3   -- NYSE/AMEX/Nasdaq
""")

permno = int(ice_permno["permno"].iloc[0])
print("ICE permno:", permno)

###########################################
# 2) Download daily stock data 2021–2023
###########################################

ice_stock = conn.raw_sql(f"""
    SELECT date, prc, ret, shrout
    FROM crsp.dsf
    WHERE permno = {permno}
      AND date BETWEEN '2021-01-01' AND '2023-12-31'
    ORDER BY date
""", date_cols=["date"])

# CRSP stores prices with sign for direction; take absolute
ice_stock["prc"] = ice_stock["prc"].abs()

ice_stock = ice_stock.rename(columns={"prc": "S"})  # S = underlying price

print(ice_stock.head())
print(ice_stock.tail())

ICE permno: 90993
        date       S       ret    shrout
0 2021-01-04   113.3 -0.017261  561284.0
1 2021-01-05  113.54  0.002118  561284.0
2 2021-01-06  115.74  0.019376  561284.0
3 2021-01-07  115.81  0.000605  561284.0
4 2021-01-08  117.63  0.015715  561284.0
          date          S       ret    shrout
748 2023-12-22     125.72  0.011994  572364.0
749 2023-12-26     126.65  0.007397  572364.0
750 2023-12-27     127.79  0.009001  572364.0
751 2023-12-28  128.60001  0.006339  572364.0
752 2023-12-29  128.42999 -0.001322  572364.0


In [7]:
ice_stock = ice_stock.set_index("date")
ice_stock = ice_stock.drop(columns=["ret", "shrout"])

In [10]:
ice_stock.to_csv("data/ice_stock_2021_2023.csv")

In [6]:
uniques = set([s[:3] for s in symbols['symbol'].tolist()])

In [18]:
import json
with open('unique_symbols.json', 'w') as f:
    json.dump(list(uniques), f)

In [21]:
query_ice = """
SELECT *
FROM optionm.opprcd2021
WHERE symbol LIKE %(pat)s
"""
ice_options = conn.raw_sql(query_ice, params={"pat": "ICE%"})


In [22]:
ice_options.to_csv('ice_options_2021.csv', index=False)